# Agentic Patterns: ReAct, Plan-Execute, and Reflection

In [1]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, create_react_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal, TypedDict, Annotated
import operator
import os

print("All imports successful")

c:\Users\ncc333\Desktop\Deep_Learning\a_Deep\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All imports successful


In [2]:
load_dotenv()
api_key = os.getenv("paid_api")

if not api_key:
    raise ValueError("API_Key not found. Please set it in your .env file")
print("API key loaded")

API key loaded


In [3]:
## Initialize LLM
llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature=0.5,
    api_key = api_key
)
print(f"LLM initialized: {llm.model_name}")

LLM initialized: gpt-4o-mini


## ReAct Pattern

## Plan-Execute Pattern

### Implementation

In [ ]:
#Customized state for plan-execute
class PlanExecuteState(TypedDict):
    """State for plan-execute pattern."""
    input: str 
    plan: list[str]
    current_step: int
    results: Annotated[list[str], operator.add] #Results from each step
    final_output: str

print("Plan-Execute state defined")

In [ ]:
# Node 1: Planner
def planner(state: PlanExecuteState) -> dict:
    """Create a step-by-step plan."""
    prompt = f"""Create a step-by-step plan for this task:

Task: {state['input']}

Return a numbered list of concrete steps. Keep it simple (3-5 steps)."""
    
    response = llm.invoke([HumanMessage(content=prompt)])

    # Parse steps (simple parsing)
    lines = response.content.split('\n')
    steps = [line.strip() for line in lines if line.strip() and any(char.isdigit() for char in line[:3])]

    print(lines)
    print(f"\n📋 PLAN CREATED:")
    for step in steps:
        print(f"    {step}")
    print()

    return {"plan": steps, "current_step": 0, "results": []}

# Node 2: Executor
def executor(state: PlanExecuteState) -> dict:
    """Execute current step."""
    if state["current_step"] >= len(state["plan"]):
        #All steps done
        return {}
    
    current_step = state["plan"][state["current_step"]] #maps current step to a planned task

    print(f"Executing: {current_step}")

    # Execute step (simplified - just use LLM)
    prompt = f"""Previous results: {state.get('results', [])}\n\nExecute this step: {current_step}"""
    response = llm.invoke([HumanMessage(content=prompt)])

    result = f"Step {state['current_step']+1} result: {response.content}"
    print(result)
    print(f"✓ Done\n")

    return {
        "results": [result],
        "current_step": state["current_step"]+1
    }

#Node Finalizer
def finalizer(state: PlanExecuteState) -> dict:
    """Create final output from all results."""
    prompt = f"""Combine these step results into final answer:

Original task: {state['input']}

Results: {state['results']}

Provide a clear, concise final answer."""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"final_output": response.content}

print("Plan-Execute nodes defined")

In [ ]:
# Routing function
def should_continue_execution(state: PlanExecuteState) -> Literal["executor", "finalizer"]:
    """Decide if more steps to execute."""
    if state["current_step"] < len(state["plan"]):
        return "executor"
    return "finalizer"

#Build Plan-Execute graph
plan_execute_builder = StateGraph(PlanExecuteState)

plan_execute_builder.add_node("planner", planner)
plan_execute_builder.add_node("executor", executor)
plan_execute_builder.add_node("finalizer", finalizer)

plan_execute_builder.add_edge(START, "planner")
plan_execute_builder.add_edge("planner", "executor")
plan_execute_builder.add_conditional_edges(
    "executor",
    should_continue_execution,
    {"executor": "executor", "finalizer": "finalizer"}
)
plan_execute_builder.add_edge("finalizer", END)
